# NREL Wave Hindacast Data Access Example

#### This notebook demonstrates basic usage of the National Renewable Energy Laboratory (NREL) West Coast Wave Hindcast dataset. The data is provided from Amazon Web Services using the HDF Group's Highly Scalable Data Service (HSDS).

#### For this example to work, the packages necessary can be installed via pip:

pip install -r requirements.txt

#### Next you'll need to configure h5pyd to access the data on HSDS:

hsconfigure

#### and enter at the prompt:

hs_endpoint = https://developer.nrel.gov/api/hsds   
hs_username = None   
hs_password = None   
hs_api_key = 3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf    

#### The example API key here is for demonstation and is rate-limited per IP. To get your own API key, visit https://developer.nrel.gov/signup/

#### You can also add the above contents to a configuration file at ~/.hscfg

#### Finally, you can use Jupyter Notebook or Lab to view the example notebooks depending on your preference


In [1]:
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Basic Usage

The Wave Hindcast Dataset is provided in annual .h5 files and currently spans the Western Coastal Region of the Lower 48 from 1979-2010.

Each year can be accessed from /nrel/us-wave/US_Wave_${year}.h5

To open the desired year of Wave Hindcast data server endpoint, username, password is found via a config file

In [2]:
waves = h5pyd.File(f'/nrel/us-wave/US_Wave_1990.h5','r')

In [ ]:
list(waves) # list of base attributes of the dataset

## Datasets

Each dataset quantity:

- directionality_coefficient
- energy_period
- maximum_energy_direction
- mean_absolute_period
- mean_zero-crossing_period
- omni-directional_wave_power
- peak_period
- significant_wave_height
- spectral_width
- water_depth

is structured as ('time_index','coordinate')


In [ ]:
# Shapes of datasets
waves['significant_water_height'].shape # (time_index, coordinates)

In [ ]:
# List the available dataset variable names within the dataset
print(list(waves['coordinates'].attrs))

In [ ]:
# Locational information is stored in either 'meta' or 'coordinates'

meta = pd.DataFrame(waves['meta'][:])
meta.head()

# Basic Usage

The following examples illustrate basic spatial and timeseries based slicing techniques for the West Coast Wave Hindcast dataset, along with simple statistical analysis. 

## Time Slicing

Basic description of timestamp extraction

In [3]:
# Extract datetime index for datasets (Saved as binary)

time_index = pd.to_datetime(waves['time_index'][:].astype(str))
time_index, time_index.inferred_freq # Temporal resolution is 3 hours

(DatetimeIndex(['1990-01-01 00:00:00+00:00', '1990-01-01 03:00:00+00:00',
                '1990-01-01 06:00:00+00:00', '1990-01-01 09:00:00+00:00',
                '1990-01-01 12:00:00+00:00', '1990-01-01 15:00:00+00:00',
                '1990-01-01 18:00:00+00:00', '1990-01-01 21:00:00+00:00',
                '1990-01-02 00:00:00+00:00', '1990-01-02 03:00:00+00:00',
                ...
                '1990-12-30 18:00:00+00:00', '1990-12-30 21:00:00+00:00',
                '1990-12-31 00:00:00+00:00', '1990-12-31 03:00:00+00:00',
                '1990-12-31 06:00:00+00:00', '1990-12-31 09:00:00+00:00',
                '1990-12-31 12:00:00+00:00', '1990-12-31 15:00:00+00:00',
                '1990-12-31 18:00:00+00:00', '1990-12-31 21:00:00+00:00'],
               dtype='datetime64[ns, UTC]', length=2920, freq=None),
 '3H')

In [4]:
Aug_7th_midnight = time_index.get_loc('1990-08-07 00:00:00') #precise data returns single index
Aug = time_index.get_loc('1990-08') #Inprecise will return a range a values

In [ ]:
print(f'August 7th midnight: {time_index[Aug_7th_midnight]}')
print(f'Month of August: {time_index[Aug]}')

## Mapping Data

Basic methods to quickly plot data from the Hindcast spatially and temporally

### Spatial Plots

Coordinate pair indices are used to slice the datasets, and are accessed through the 'coordinates' attribute

In [7]:
coord_slice = 100

print(f'Available attributes: {dict(waves["coordinates"].attrs)}')
coords = waves['coordinates'][::coord_slice]
print(f'Coordinate pairs: {coords}')

Available attributes: {'description': '(latitude, longitude) using Datum: NAD83', 'dimensions': array([array('position', dtype='<U8')], dtype=object), 'src_name': '(Xp, Yp)', 'units': '(deg, deg)'}
Coordinate pairs: [[  48.8641 -125.386 ]
 [  46.1511 -129.577 ]
 [  41.74   -129.538 ]
 ...
 [  37.4535 -122.072 ]
 [  37.4517 -122.055 ]
 [  37.4617 -122.027 ]]


In [8]:
# Building a scatter plot of significant wave height values

df = pd.DataFrame() # Combine data with coordinates in a DataFrame
df['longitude'] = coords[:, 1]
df['latitude'] = coords[:, 0]
df['Hsig'] = waves['significant_water_height'][Aug_7th_midnight, ::coord_slice] 

In [9]:
%matplotlib widget

df['Hsig'][df['Hsig']<0] = np.nan # There are negative values here Levi
df.plot.scatter(x='longitude', y='latitude', c='Hsig',
                colormap='viridis',
                title=str(time_index[Aug_7th_midnight]))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Timeseries Plots

Extracting a timeseries usually requires us to find a location of interest.      
This can be done easially with KDTree, a function within scipy.spatial.   


In [10]:
from scipy.spatial import cKDTree

tree = cKDTree(coords)
def nearest_site(tree, lat_coord, lon_coord):
    lat_lon = np.array([lat_coord, lon_coord])
    dist, pos = tree.query(lat_lon)
    return pos

lat_coord, lon_coord = 39.6122, -125.318
site_idx = nearest_site(tree, lat_coord, lon_coord)

print(f'Site index: {site_idx}')
print(f'Coordinates Searched: {lat_coord, lon_coord}')
print(f'Coordinates of Nearest Point: {coords[site_idx]}')

Site index: 2567
Coordinates Searched: (39.6122, -125.318)
Coordinates of Nearest Point: [  39.546 -125.174]


In [11]:
# Now we can extract data at that location over a specified time range

var = 'peak_period'

dt = pd.DataFrame(waves[var][Aug,site_idx],index=time_index[Aug],columns=[var])
dt.plot(y=var,title=f'Time sliced {var} Timeseries')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Basic Statistics and Grouping

Here is an example of grouping data to create quick plots of statistics

In [54]:

def round_index(df):
    name = df.index.name
    df.set_index(np.round(df.index.values,1),inplace=True) 
    df.index.name = f'{name} (S)'
    return df

Vars =['significant_water_height','peak_period'] 
variables = np.array([waves[Vars[0]][Aug,site_idx], waves[Vars[1]][Aug,site_idx]])

ds = pd.DataFrame(variables.T,
                    index=time_index[Aug],
                    columns=Vars
                )
group = ds.groupby('peak_period')

In [55]:
# Plot the number of occurances of sea-states within peak period bins

count = group.count()
count = round_index(count)

count.plot(kind='bar',rot=45)
plt.tight_layout()


/home/abharath/.envs/py36/lib/python3.6/site-packages/pandas/plotting/_matplotlib/core.py:320: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
# plot the mean Hsig of the sea-states within peak period bins along with
# the variance of those waveheights

Mean = group.mean()
Var = group.var()
Mean = round_index(Mean)
Var = round_index(Var)

Mean.plot(kind='bar',yerr=Var)
plt.tight_layout()

/home/abharath/.envs/py36/lib/python3.6/site-packages/pandas/plotting/_matplotlib/core.py:320: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
plt.close()